<a href="https://colab.research.google.com/github/jedondata/handsongenai/blob/main/Exercises/5-NewsClassification_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [38]:
!pip install textblob 'keras-nlp' 'keras-preprocessing' 'gensim==4.2.0' 'tensorflow_text==2.15' np_utils swifter

# add 'tensorflow_text==2.15' for compatibility

In [6]:
import multiprocessing
import tensorflow as tf
import sys
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda, ELU, Conv1D, MaxPooling1D, Dropout
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from textblob import TextBlob, Word
from keras_preprocessing.sequence import pad_sequences
from keras.initializers import Constant
import numpy as np
import random
import os
import pandas as pd
import gensim
import warnings
import nltk
import pickle
from tensorflow.nn import leaky_relu

import re
import warnings
from sklearn.model_selection import train_test_split
from textblob import TextBlob

TRACE = False
embedding_dim = 300
epochs=100
batch_size = 250
corpus_size=25000
BATCH = True

def set_seeds_and_trace():
  os.environ['PYTHONHASHSEED'] = '0'
  np.random.seed(42)
  tf.random.set_seed(42)
  random.seed(42)
  if TRACE:
    tf.debugging.set_log_device_placement(True)

def set_session_with_gpus_and_cores():
  cores = multiprocessing.cpu_count()
  gpus = len(tf.config.list_physical_devices('GPU'))
  config = tf.compat.v1.ConfigProto( device_count = {'GPU': gpus  , 'CPU': cores} , intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(config=config)
  tf.compat.v1.keras.backend.set_session(sess)

set_seeds_and_trace()
set_session_with_gpus_and_cores()
warnings.filterwarnings('ignore')
nltk.download('punkt')
textblob_tokenizer = lambda x: TextBlob(x).words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
%%writefile get_data.sh
if [ ! -f news.csv ]; then
  wget -O news.csv https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
fi

Writing get_data.sh


In [8]:
!bash get_data.sh

--2024-05-14 13:14:11--  https://www.dropbox.com/s/352x7xzivf60zgc/news.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/352x7xzivf60zgc/news.csv [following]
--2024-05-14 13:14:11--  https://www.dropbox.com/s/raw/352x7xzivf60zgc/news.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc379e339c2bc0283e49b4e79a84.dl.dropboxusercontent.com/cd/0/inline/CS4ZqqOzE39NgM6Kwp7nzMVNVakvMHd4H46j0n4IvGGsQOjXiSa5eafBARccJTVxAofI8ZbyoPNhXFl--Eemf2rCsDlygPJB5lWLbax-lARispZWgU9nnkmLthyi_VB_BzUinAJXJsdpVpEGvwUfUv-f/file# [following]
--2024-05-14 13:14:11--  https://uc379e339c2bc0283e49b4e79a84.dl.dropboxusercontent.com/cd/0/inline/CS4ZqqOzE39NgM6Kwp7nzMVNVakvMHd4H46j0n4IvGGsQOjXiSa5eafBARccJTVxAofI8ZbyoPNhXFl--Eemf2rCsDlygPJB5lWLbax-lARi

In [9]:
path = './news.csv'
news_pre = pd.read_csv(path, header=0).sample(n=corpus_size).reset_index(drop=True)

def preprocess_text(text, should_join=True):
    # Use the tokenizer to tokenize into words, lowercase them, remove punctuation, and finally use gensim.utils.simple_preprocess(text)
    text = None
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

In [39]:
import gensim

def preprocess_text(text, should_join=True):
    # Tokenize into words, lowercase them, remove punctuation, and use gensim.utils.simple_preprocess
    text = ' '.join([word.lower() for word in textblob_tokenizer(text)])
    if should_join:
        return ' '.join(gensim.utils.simple_preprocess(text))
    else:
        return gensim.utils.simple_preprocess(text)


In [42]:

preprocess_text('This is the best party ever! Right ?!')

'this is the best party ever right'

In [13]:
import re
nltk.download('stopwords')
my_stopwords = nltk.corpus.stopwords.words('english')
word_rooter = nltk.stem.snowball.PorterStemmer(ignore_stopwords=False).stem
my_punctuation = '!"$%&\()*+,-./:;<=>?[\\]^_`{|}~•@'


def preprocess_text(text, should_join=True):
    text = ' '.join(word.lower() for word in textblob_tokenizer(text))
    text = re.sub(r'http\S+', '', text) # remove http links
    text = re.sub(r'bit.ly/\S+', '', text) # rempve bitly links
    text = text.strip('[link]') # remove [links]
    text = re.sub('['+my_punctuation + ']+', ' ', text) # remove punctuation
    text = re.sub('\s+', ' ', text) #remove double spacing
    text = re.sub(r"[^a-zA-Z.,&!?]+", r" ", text) # only normal characters
    text_token_list = [word for word in text.split(' ')
                            if word not in my_stopwords] # remove stopwords
    text_token_list = [word_rooter(word) if '#' not in word else word
                        for word in text_token_list] # apply word rooter
    text = ' '.join(text_token_list)
    if should_join:
      return ' '.join(gensim.utils.simple_preprocess(text))
    else:
      return gensim.utils.simple_preprocess(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
import swifter
# Use swifter to apply the preprocessin and save that pandas series to a file
news = news_pre.title.swifter.apply(preprocess_text)

# Save the preprocessed text to a CSV file
news.to_csv('news_preprocessed.csv', index=False, header=False)

Pandas Apply:   0%|          | 0/25000 [00:00<?, ?it/s]

In [47]:


class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        corpus_path = 'news_preprocessed.csv'
        for line in open(corpus_path):
            # assume there's one document per line, tokens separated by whitespace
            yield preprocess_text(line, should_join=False)


from gensim.models import Word2Vec  # Ensure this import is present
corpus = MyCorpus()
word2vec_model = Word2Vec(sentences=corpus, vector_size=embedding_dim, window=5, min_count=1, workers=multiprocessing.cpu_count())
# Get a word2vec model using gensim.models and passing the sentences using MyCorpus()
# min_count = 1 or 2, 3 is a lot



In [48]:
# checking the wv

wv_model = word2vec_model.wv

In [49]:
weights = wv_model.vectors  # Get the weights of the model (the embedding) and convert to tensor. Hint: Check word2vec_model.wv
vocab_size = len(wv_model.index_to_key)  # get voc  ab size from index_to_key in word2vec_model.wv



In [52]:
# key to index

wv_model.key_to_index

{'to': 0,
 'in': 1,
 'for': 2,
 'on': 3,
 'of': 4,
 'ap': 5,
 'the': 6,
 'gt': 7,
 'lt': 8,
 'reuters': 9,
 'at': 10,
 'with': 11,
 'us': 12,
 'new': 13,
 'as': 14,
 'up': 15,
 'and': 16,
 'says': 17,
 'oil': 18,
 'over': 19,
 'by': 20,
 'iraq': 21,
 'microsoft': 22,
 'is': 23,
 'out': 24,
 'from': 25,
 'after': 26,
 'update': 27,
 'afp': 28,
 'deal': 29,
 'no': 30,
 'china': 31,
 'win': 32,
 'may': 33,
 'more': 34,
 'it': 35,
 'be': 36,
 'talks': 37,
 'stocks': 38,
 'profit': 39,
 'bush': 40,
 'game': 41,
 'world': 42,
 'off': 43,
 'will': 44,
 'first': 45,
 'wins': 46,
 'back': 47,
 'not': 48,
 'prices': 49,
 'sales': 50,
 'ibm': 51,
 'google': 52,
 'com': 53,
 'eu': 54,
 'security': 55,
 'report': 56,
 'down': 57,
 'top': 58,
 'open': 59,
 'into': 60,
 'india': 61,
 'against': 62,
 'japan': 63,
 'two': 64,
 'red': 65,
 'space': 66,
 'takes': 67,
 'un': 68,
 'end': 69,
 'set': 70,
 'rise': 71,
 'court': 72,
 'team': 73,
 'record': 74,
 'year': 75,
 'but': 76,
 'high': 77,
 'killed': 

In [53]:
weights.shape

(17537, 300)

In [54]:
news = pd.read_csv('news.csv')

# Print the column names to find out the correct name for the title column
print(news.columns)



Index(['category', 'title'], dtype='object')


In [55]:
news_preprocessed = pd.DataFrame()
news_preprocessed['label'] = news_pre.category.map({'Business': 0, 'Sports': 1, 'Sci/Tech': 2, 'World': 3})
news_preprocessed['title'] = news['title']
news_preprocessed

,label,title
0,0,Wall St. Bears Claw Back Into the Black (Reuters)
1,0,Carlyle Looks Toward Commercial Aerospace (Reu...
2,1,Oil and Economy Cloud Stocks' Outlook (Reuters)
3,2,Iraq Halts Oil Exports from Main Southern Pipe...
4,0,"Oil prices soar to all-time record, posing new..."
...,...,...
24995,0,Terrapins too tough
24996,2,"There's no stopping McLaughlin, Woburn"
24997,1,It's no easy task for Notre Dame
24998,1,Tough trio intends to stick it out


In [58]:
def get_maximum_review_length(df):
    maximum = 0
    for ix, row in df.iterrows():
        candidate = len(textblob_tokenizer(row.title))
        if candidate > maximum:
            maximum = candidate
    return maximum


maximum = get_maximum_review_length(news_preprocessed)   # Since 2 titles may have different number of words, we have to find the max length and fill with 0s if a title is shorter
maximum

23

In [75]:
X = np.zeros((len(news_preprocessed), maximum))   # Here we do what we said above
# Iterate through the news df and for every word, if it exists in the word2vec model, put into X for that review and that word the index of the embedding (check index_to_key)
# HINT: to iterate through a column of a pandas dataframe you do:

# Most manual way - basic of NLP process
for row_index, row in news_preprocessed.iterrows():
    # Split the preprocessed text into words
    for column_index, word in enumerate(textblob_tokenizer(row.title)):
      if word in wv_model.key_to_index:
          word_id = wv_model.key_to_index[word]
      else:
          word_id = 0

      X[row_index, column_index] = word_id



# for index, value in df.iterrows():
#      #do something

# FILL
y = news_preprocessed.label


In [76]:
X[0:15].astype(int)

array([[    0,     0,     0,     0,     0,     0,     6,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,    16,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,    25,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,    49,  1583,     0,     0,    74, 17142,    13,  9536,
            0,     0,   260,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0],
       [    0,     0,     0,     0,     0,     0,     0,     0,     0,

In [81]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = tf.constant(X_train)
X_test = tf.constant(X_test)
# Convert y_train and y_test from an array of values between 0-3 to a one hot matrix tensor
y_train = tf.one_hot(y_train, depth =4)
y_test = tf.one_hot(y_test, depth = 4)


In [82]:
y_train[:5]

<tf.Tensor: shape=(5, 4), dtype=float32, numpy=
array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)>

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, trainable = False, input_lenth = maximum, weights = [weights]))  # Add an Embedding layer with weights being the rweights variable and trainable as False. The embedding dimension should be embedding_dim
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu')) # Add a couple of Dense Layers with RELU or leaky_relu activations. You may add Batch Norm if you want too
model.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(None, embedding_dim,)))  # Average out the words of the sentence. The expected out is (N, D) where N is number f samples in batch and D is embedding dimension
model.add()  # Add final Dense layer

In [83]:
# Compile the model. Think what is the best loss to use
model.compile(loss = 'categorical_crossentropy')
model.summary()

NameError: name 'model' is not defined

In [84]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.01)
history = None # Fit the model, use the callback above to do EarlyStopping

In [85]:
import matplotlib.pyplot as plt

# function for plotting loss
def plot_metrics(train_metric, val_metric=None, metric_name=None, title=None, ylim=5):
    plt.title(title)
    plt.ylim(0,ylim)
    plt.plot(train_metric,color='blue',label=metric_name)
    if val_metric is not None: plt.plot(val_metric,color='green',label='val_' + metric_name)
    plt.legend(loc="upper right")

# plot loss history
plot_metrics(history.history['loss'], history.history['val_loss'], "Loss", "Loss", ylim=2.0)


AttributeError: 'NoneType' object has no attribute 'history'

In [ ]:
# Test with the following two sentences:
# - 'supercomputer will put workers jobless soon'
# - 'patriots goes winning super bowl'

x_val = np.zeros((2, maximum))
for index, row in enumerate(['supercomputer will put workers jobless soon', 'patriots goes winning super bowl']):
    if word in wv_mode.key_to_index:
      word_id = wv_model.key_to_index[word]
    else:
      word_id = 0
    x_val[index, column_index]
y_val = tf.one_hot([0,1], depth=4)